
## TWL and Wind (speed & direction ) for swash simulations

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.climate_emulator import Climate_Emulator
from teslakit.waves import AWL, Aggregate_WavesFamilies, Intradaily_Hydrograph
from teslakit.util.time_operations import xds_limit_dates, repair_times_hourly, add_max_storms_mask

from teslakit.io.aux_nc import StoreBugXdset


In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'

db = Database(p_data)

# set site
site = 'KWAJALEIN_emulator'
db.SetSite(site)


## Save offshore TWL (AT + MMSL + SSp)

In [3]:
# offshore level 
offshore_vars = db.Load_SIM_OFFSHORE_all(vns=['AT', 'MMSL', 'SS', 'wind_speed', 'wind_dir', 'wind_dir', 'mslp'], decode_times=True, use_cftime=True)

# mslp from Pa to mb
offshore_vars['mslp'] = offshore_vars.mslp*0.01

print(offshore_vars)
print()



<xarray.Dataset>
Dimensions:     (n_sim: 10, time: 8765832)
Coordinates:
  * time        (time) object 2000-01-01 00:00:00 ... 2999-12-31 23:00:00.13
  * n_sim       (n_sim) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    AT          (time, n_sim) float64 ...
    MMSL        (time, n_sim) float64 ...
    SS          (time, n_sim) float64 ...
    wind_speed  (time, n_sim) float64 ...
    wind_dir    (time, n_sim) float64 ...
    mslp        (time, n_sim) float64 1.011e+03 1.01e+03 ... 1.01e+03 1.011e+03



In [4]:
# historical SLP
press = db.Load_ESTELA_SLP() # Pa     

# Kwajalein: pnt_lon = 167.73, pnt_lat = 8.72
lonp = 168 # closest coordinates in SLP dataset
latp = 8

# select closest grid point to Site
press = press.sel(longitude = lonp, latitude = latp) *0.01 # from Pa to mb
print(press)
print()

# mean value
mean_press = np.nanmean(press.SLP.values)
print(mean_press)


<xarray.Dataset>
Dimensions:    (time: 359401)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 ... 2020-01-01
    latitude   float32 8.0
    longitude  float32 168.0
Data variables:
    SLP        (time) float64 1.007e+03 1.007e+03 ... 1.013e+03 1.012e+03

1009.5448758639812


In [5]:
# SS de barómetro inverso
offshore_vars['IB'] = -1*(offshore_vars['mslp'] - mean_press) # cm
offshore_vars['IB'] = offshore_vars['IB']*0.01 # m

# donde SS = 0 (no TC) poner SS de barómetro inverso
TWL = offshore_vars.copy(deep=True)
TWL['SS'] = TWL['SS'].where(TWL['SS']!=0, TWL['IB']) 

    

In [6]:
# Save TWL
print(TWL)

TWL['TWL'] = TWL.AT + TWL.MMSL + TWL.SS

TWL = TWL.drop({'AT', 'MMSL', 'SS', 'IB', 'mslp'})
print(TWL)




<xarray.Dataset>
Dimensions:     (n_sim: 10, time: 8765832)
Coordinates:
  * time        (time) object 2000-01-01 00:00:00 ... 2999-12-31 23:00:00.13
  * n_sim       (n_sim) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    AT          (time, n_sim) float64 0.2456 0.2456 0.2456 ... -0.6179 -0.6179
    MMSL        (time, n_sim) float64 0.009233 0.06078 ... -0.07514 -0.1264
    SS          (time, n_sim) float64 -0.009747 -0.003241 ... -0.008799 -0.01767
    wind_speed  (time, n_sim) float64 9.392 5.381 6.011 ... 7.533 13.99 4.628
    wind_dir    (time, n_sim) float64 57.1 88.07 35.95 ... 120.5 62.91 69.68
    mslp        (time, n_sim) float64 1.011e+03 1.01e+03 ... 1.01e+03 1.011e+03
    IB          (time, n_sim) float64 -0.009747 -0.003241 ... -0.008799 -0.01767
<xarray.Dataset>
Dimensions:     (n_sim: 10, time: 8765832)
Coordinates:
  * time        (time) object 2000-01-01 00:00:00 ... 2999-12-31 23:00:00.13
  * n_sim       (n_sim) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    wind_speed  

In [ ]:
# Save
StoreBugXdset(TWL, '/Users/albacid/Projects/TeslaKit_projects/sites/' + site + '/SIMULATION/OFFSHORE/offshore_vars.nc')
